In [1]:
from IPython.display import HTML
import re
import os
import shutil
from bs4 import BeautifulSoup
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [37]:
import run
s = '''# BEGIN PROB

Let's learn about this problem.

# BEGIN SUBPROB

This should work.

( ) Hey!
( ) How are you?

So should this:

[ ] $f(x^2) + g(x^2)$
[ ] `lambda x: x ** 2`
[ ] another option

So should this!

# BEGIN SOLUTION

$$L(x) = f(x) + f(x^2) + f(x^3) + ...$$

<center><iframe width="560" height="315" src="https://www.youtube.com/embed/enRxW-djG58" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>

# END SOLUTION

# END SUBPROB

# BEGIN SUBPROB

<center><img src='../assets/images/wi22-final/plum.png' width=30%></center>

Here's an image. Recreate that DataFrame.

[ ] Write the press release
[ ] Other option

# BEGIN SOLUTION

Here's how you'd do it:

```java

public static void main(String[] args){
    System.out.println("hello, world!");
}
```

<center><img src='../assets/images/sp22-midterm/graph-paper-soln.png' width=330></center>

Is there something weird about $\log(f(x)) + \frac{2}{4}$? How about

$$\int_\infty^\infty [f(x) + g(x^2)] dx$$

# END SOLUTION

# END SUBPROB

# END PROB'''

out = run.process_problem(s, 1, show_solution=True)

HTML(out)

In [ ]:
print(re.sub(r'\n\n([[(] [])] [\d\D]+?)\n\n', '15', s))

In [ ]:
def delete_folder(path):
    '''Needed for Windows...
       Taken from https://stackoverflow.com/questions/21261132/shutil-rmtree-to-remove-readonly-files
    '''

    def del_rw(action, name, exc):
        os.chmod(name, stat.S_IWRITE)
        os.remove(name)

    return shutil.rmtree(path, onerror=del_rw)

In [ ]:
def pandoc(s, kind='md', flags=''):
    '''Take in a string containing Markdown and return its HTML equivalent, via pandoc'''
    assert kind == 'tex' or kind == 'md', 'kind must be tex or md'

    if not os.path.exists('temp'):
        os.mkdir('temp')

    in_path = os.path.join('temp', f'temp.{kind}')
    in_file = open(in_path, 'w')
    in_file.write(s)
    in_file.close()

    src_path = os.path.join('temp', f'temp.{kind}')
    dst_path = os.path.join('temp', 'temp.html')
    os.system(f'pandoc -s --standalone --katex --from markdown-markdown_in_html_blocks+raw_html --metadata title=" " -s {src_path} {flags} -o {dst_path}')

    out_path = os.path.join('temp', 'temp.html')
    out_file = open(out_path, 'r')
    out_s = out_file.read()
    out_file.close()
    delete_folder('temp')

    soup = BeautifulSoup(out_s, features='lxml')
    return str(soup.find('body')).replace('<body>', '').replace('</body>', '')

In [ ]:
def process_MC_matches(matchobj):
    '''Helper function for process_MC'''
    match_str = matchobj.group()
    
    # Multiple choice (circular boxes)
    if match_str.count('( )') >= 2:
        exp = r'\( \) (.*)'
        input_type = 'radio'
        
    # Select all (square boxes)
    elif match_str.count('[ ]') >= 2:
        exp = r'\[ \] (.*)'
        input_type = 'checkbox'
        
    else:
        # print(match_str)
        # print('match_str.count("( )") = ', match_str.count('( )'))
        # print('match_str.count("[ ]") = ', match_str.count('[ ]'))
        raise SyntaxError('How did this happen?')
        
    choices = re.findall(exp, match_str)
    out = '\n\n<ul class="task-list">\n'
    for choice in choices:
        processed_choice = pandoc(choice) # In case the choice includes Markdown
        processed_choice = str(BeautifulSoup(processed_choice, features='lxml').find('p')).replace('<p>', '').replace('</p>', '')
        out += f'<li><p><input type="{input_type}" disabled="" /> {processed_choice}</p></li>\n'
    
    out += '</ul>\n\n'
    
    return out

def process_MC(problem_str):
    '''Critical assumption: any problem with multiple choice or select all options has at least 2 choices'''
    
    extract_exp = '\n\n([[(] [])] [\d\D]+?)\n\n' # TODO experiment with number of \n here
    print('---\n', problem_str, '\n---')
    return re.sub(extract_exp, process_MC_matches, problem_str)